# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To reach the other side, perchance to find reprieve from thy jesting.


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa! It's nice to meet you. How are you doing today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to personal information like your name. Feel free to tell me your name, though, and I'll do my best to remember it during our conversation.


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [10]:
%pip install -q -U panel

Note: you may need to restart the kernel to use updated packages.


In [11]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [12]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

/var/folders/pl/z799lglj00b41cclm26nwxmh0000gn/T/ipykernel_34030/198781390.py:2: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [13]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [14]:
# VERSION 1: Résumé de commande avec format JSON structuré et calcul automatique des prix
print("=== VERSION 1: Format JSON Structuré ===")
messages_v1 = context.copy()
messages_v1.append(
{'role':'system', 'content':'Create a detailed JSON summary of the food order. Calculate prices automatically and format as follows:\
{\
  "order_summary": {\
    "pizzas": [{"type": "", "size": "", "unit_price": 0.00, "quantity": 1}],\
    "toppings": [{"name": "", "unit_price": 0.00, "quantity": 1}],\
    "drinks": [{"type": "", "size": "", "unit_price": 0.00, "quantity": 1}],\
    "sides": [{"type": "", "size": "", "unit_price": 0.00, "quantity": 1}],\
    "pricing": {\
      "subtotal": 0.00,\
      "tax": 0.00,\
      "total": 0.00\
    },\
    "order_type": "pickup/delivery",\
    "estimated_time": "XX minutes"\
  }\
}'},
)

response_v1 = get_completion_from_messages(messages_v1, temperature=0)
print(response_v1)
print("\n" + "="*60 + "\n")

# VERSION 2: Résumé de commande style reçu de restaurant avec émojis
print("=== VERSION 2: Style Reçu de Restaurant avec Émojis ===")
messages_v2 = context.copy()
messages_v2.append(
{'role':'system', 'content':'Create a fun restaurant receipt-style summary with emojis. Format it like this:\
🍕 PIZZA PALACE RECEIPT 🍕\
════════════════════════════\
📅 Date: [current date]\
⏰ Time: [current time]\
👤 Order for: Customer\
\
📋 ORDER DETAILS:\
🍕 Pizzas:\
   - [pizza details with emojis]\
🧄 Toppings:\
   - [toppings with prices]\
🥤 Drinks:\
   - [drinks with sizes and prices]\
🍟 Sides:\
   - [sides with details]\
\
💰 PRICING BREAKDOWN:\
   Subtotal: $XX.XX\
   Tax (8%): $X.XX\
   Total: $XX.XX\
\
📦 Order Type: [pickup/delivery]\
⏱️ Ready in: XX minutes\
\
Thank you for choosing Pizza Palace! 🍕❤️'},
)

response_v2 = get_completion_from_messages(messages_v2, temperature=0)
print(response_v2)
print("\n" + "="*60 + "\n")

# VERSION 3: Résumé de commande style tableau avec analyse nutritionnelle
print("=== VERSION 3: Tableau avec Analyse Nutritionnelle ===")
messages_v3 = context.copy()
messages_v3.append(
{'role':'system', 'content':'Create a comprehensive order summary in table format with nutritional estimates. Include:\
\
ORDER SUMMARY TABLE:\
| Item | Size | Qty | Unit Price | Total Price |\
|------|------|-----|------------|-------------|\
[populate table rows]\
\
NUTRITIONAL ESTIMATE (approximate):\
- Total Calories: ~XXX kcal\
- Protein: ~XXg\
- Carbohydrates: ~XXg\
- Fat: ~XXg\
- Sodium: ~XXmg\
\
ALLERGEN INFORMATION:\
- Contains: [list potential allergens]\
- Gluten-free options: [yes/no]\
\
COST BREAKDOWN:\
- Food Subtotal: $XX.XX\
- Tax (8%): $X.XX\
- Delivery Fee: $X.XX (if applicable)\
- Total Amount: $XX.XX\
\
PREPARATION TIME: XX-XX minutes\
ORDER TYPE: [pickup/delivery]'},
)

response_v3 = get_completion_from_messages(messages_v3, temperature=0)
print(response_v3)
print("\n" + "="*60 + "\n")

print("🎯 ANALYSE DES 3 VERSIONS:")
print("Version 1: Format JSON structuré - idéal pour l'intégration API")
print("Version 2: Style reçu avec émojis - expérience utilisateur amusante")
print("Version 3: Tableau avec nutrition - information complète et santé")

=== VERSION 1: Format JSON Structuré ===
{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


=== VERSION 2: Style Reçu de Restaurant avec Émojis ===
{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


=== VERSION 2: Style Reçu de Restaurant avec Émojis ===
{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "t